# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 10:08AM,256687,10,0086358996,ISDIN Corporation,Released
1,Feb 6 2023 10:08AM,256687,10,0086359002,ISDIN Corporation,Released
2,Feb 6 2023 10:08AM,256687,10,0086359010,ISDIN Corporation,Released
3,Feb 6 2023 10:08AM,256687,10,0086359014,ISDIN Corporation,Released
4,Feb 6 2023 9:45AM,256685,21,640397,"NBTY Global, Inc.",Released
5,Feb 6 2023 9:44AM,256684,21,640394,"NBTY Global, Inc.",Released
6,Feb 6 2023 9:44AM,256683,21,640393,"NBTY Global, Inc.",Released
7,Feb 6 2023 9:29AM,256682,10,Clear-9123589,"ClearSpec, LLC",Released
8,Feb 6 2023 9:21AM,256681,16,Em-BenHotel20623,"Emersa Waterbox, LLC",Released
9,Feb 6 2023 9:21AM,256681,16,Em-Edition20623,"Emersa Waterbox, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,256682,Released,1
20,256683,Released,1
21,256684,Released,1
22,256685,Released,1
23,256687,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256682,NaN,1.0
256683,NaN,1.0
256684,NaN,1.0
256685,NaN,1.0
256687,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256651,1.0,0.0
256656,1.0,2.0
256662,0.0,1.0
256668,1.0,6.0
256669,35.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256651,1,0
256656,1,2
256662,0,1
256668,1,6
256669,35,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256651,1,0
1,256656,1,2
2,256662,0,1
3,256668,1,6
4,256669,35,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256651,1,
1,256656,1,2
2,256662,,1
3,256668,1,6
4,256669,35,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 10:08AM,256687,10,ISDIN Corporation
4,Feb 6 2023 9:45AM,256685,21,"NBTY Global, Inc."
5,Feb 6 2023 9:44AM,256684,21,"NBTY Global, Inc."
6,Feb 6 2023 9:44AM,256683,21,"NBTY Global, Inc."
7,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC"
8,Feb 6 2023 9:21AM,256681,16,"Emersa Waterbox, LLC"
13,Feb 6 2023 9:12AM,256679,10,"Snap Medical Industries, LLC"
16,Feb 6 2023 9:12AM,256680,10,Emerginnova
18,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation
20,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 6 2023 10:08AM,256687,10,ISDIN Corporation,,4
1,Feb 6 2023 9:45AM,256685,21,"NBTY Global, Inc.",,1
2,Feb 6 2023 9:44AM,256684,21,"NBTY Global, Inc.",,1
3,Feb 6 2023 9:44AM,256683,21,"NBTY Global, Inc.",,1
4,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC",,1
5,Feb 6 2023 9:21AM,256681,16,"Emersa Waterbox, LLC",2,3
6,Feb 6 2023 9:12AM,256679,10,"Snap Medical Industries, LLC",,3
7,Feb 6 2023 9:12AM,256680,10,Emerginnova,,2
8,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation,,2
9,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation,,31


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 10:08AM,256687,10,ISDIN Corporation,4,
1,Feb 6 2023 9:45AM,256685,21,"NBTY Global, Inc.",1,
2,Feb 6 2023 9:44AM,256684,21,"NBTY Global, Inc.",1,
3,Feb 6 2023 9:44AM,256683,21,"NBTY Global, Inc.",1,
4,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC",1,
5,Feb 6 2023 9:21AM,256681,16,"Emersa Waterbox, LLC",3,2
6,Feb 6 2023 9:12AM,256679,10,"Snap Medical Industries, LLC",3,
7,Feb 6 2023 9:12AM,256680,10,Emerginnova,2,
8,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation,2,
9,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation,31,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 10:08AM,256687,10,ISDIN Corporation,4,
1,Feb 6 2023 9:45AM,256685,21,"NBTY Global, Inc.",1,
2,Feb 6 2023 9:44AM,256684,21,"NBTY Global, Inc.",1,
3,Feb 6 2023 9:44AM,256683,21,"NBTY Global, Inc.",1,
4,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 10:08AM,256687,10,ISDIN Corporation,4.0,NaN
1,Feb 6 2023 9:45AM,256685,21,"NBTY Global, Inc.",1.0,NaN
2,Feb 6 2023 9:44AM,256684,21,"NBTY Global, Inc.",1.0,NaN
3,Feb 6 2023 9:44AM,256683,21,"NBTY Global, Inc.",1.0,NaN
4,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 10:08AM,256687,10,ISDIN Corporation,4.0,0.0
1,Feb 6 2023 9:45AM,256685,21,"NBTY Global, Inc.",1.0,0.0
2,Feb 6 2023 9:44AM,256684,21,"NBTY Global, Inc.",1.0,0.0
3,Feb 6 2023 9:44AM,256683,21,"NBTY Global, Inc.",1.0,0.0
4,Feb 6 2023 9:29AM,256682,10,"ClearSpec, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3080105,229.0,36.0
16,513343,4.0,2.0
19,513332,4.0,1.0
21,770052,3.0,0.0
22,256651,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3080105,229.0,36.0
1,16,513343,4.0,2.0
2,19,513332,4.0,1.0
3,21,770052,3.0,0.0
4,22,256651,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,229.0,36.0
1,16,4.0,2.0
2,19,4.0,1.0
3,21,3.0,0.0
4,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,229.0
1,16,Released,4.0
2,19,Released,4.0
3,21,Released,3.0
4,22,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,21,22
Status,,,,,
Executing,36.0,2.0,1.0,0.0,1.0
Released,229.0,4.0,4.0,3.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,21,22
0,Executing,36.0,2.0,1.0,0.0,1.0
1,Released,229.0,4.0,4.0,3.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,21,22
0,Executing,36.0,2.0,1.0,0.0,1.0
1,Released,229.0,4.0,4.0,3.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()